In [2]:
import torch
import numpy as np
import os
import torch.neuron
import blink.main_dense as main_dense
import argparse
import json 
import time

In [4]:
#models_path = "../models/" # the path where you stored the BLINK models
models_path = "/home/ubuntu/BLINK/torchserve_sandbox/model_support"#'/opt/models/model-support/'#"model_store/" # the path where you stored the BLINK models

config = {
    "test_entities": None,
    "test_mentions": None,
    "interactive": False,
    "top_k": 10,
    "biencoder_model": models_path+"/biencoder_wiki_large.bin",
    "biencoder_config": models_path+"/biencoder_wiki_large.json",
    "entity_catalogue": models_path+"/entity.jsonl",
    "entity_encoding": models_path+"/all_entities_large.t7",
    "crossencoder_model": models_path+"/crossencoder_wiki_large.bin",
    "crossencoder_config": models_path+"/crossencoder_wiki_large.json",
    "fast": True, # set this to be true if speed is a concern
    "output_path": "logs/" # logging directory
}

args = argparse.Namespace(**config)

data_to_link = [ {
                    "id": 0,
                    "label": "unknown",
                    "label_id": -1,
                    "context_left": "".lower(),
                    "mention": "Shakespeare".lower(),
                    "context_right": "'s account of the Roman general Julius Caesar's murder by his friend Brutus is a meditation on duty.".lower(),
                },
                {
                    "id": 1,
                    "label": "unknown",
                    "label_id": -1,
                    "context_left": "Shakespeare's account of the Roman general".lower(),
                    "mention": "Julius Caesar".lower(),
                    "context_right": "'s murder by his friend Brutus is a meditation on duty.".lower(),
                }
                ]

In [5]:
models = main_dense.load_models(args, logger=None)

biencoder, biencoder_params, crossencoder, crossencoder_params, candidate_encoding, title2id, id2title, id2text, wikipedia_id2local_id, faiss_indexer = models


cuda


In [6]:
cand_encs = torch.load('sample_input/cand_encs.pt')
text_vecs = torch.load('sample_input/text_vecs.pt')

In [7]:
cand_vecs = None 
random_negs = True
model = biencoder
inputs = [text_vecs, cand_encs]

In [8]:
x=model(*inputs)
x

tensor([[71.5984, 72.7923, 82.1700, 73.7910, 67.3760],
        [72.4845, 84.5333, 72.7498, 68.8741, 63.2799]], device='cuda:0',
       grad_fn=<MmBackward>)

In [9]:
start = time.time()

_, _, _, _, _, predictions, scores, mention_found= main_dense.run(args, None, *models, test_data=data_to_link, REL_filter=True)
print(predictions)

end = time.time()
print(end - start)

100%|██████████| 1/1 [00:00<00:00, 29.54it/s][['William Shakespeare', 'Romeo and Juliet'], ['Julius Caesar', 'William Shakespeare']]
0.11205911636352539



In [10]:
module = torch.jit.trace(model, example_inputs=inputs)
module.save("blink_entity_linking.pt")

In [11]:
module_loaded=torch.jit.load("blink_entity_linking.pt")

In [12]:
x=module_loaded(*inputs)
x

tensor([[71.5984, 72.7923, 82.1700, 73.7910, 67.3760],
        [72.4845, 84.5333, 72.7498, 68.8741, 63.2799]], device='cuda:0',
       grad_fn=<MmBackward>)

In [13]:
models_jit = module_loaded, biencoder_params, crossencoder, crossencoder_params, candidate_encoding, title2id, id2title, id2text, wikipedia_id2local_id, faiss_indexer 

In [14]:
start = time.time()

_, _, _, _, _, predictions, scores, mention_found= main_dense.run(args, None, *models_jit, test_data=data_to_link, REL_filter=True)
print(predictions)

end = time.time()
print(end - start)

100%|██████████| 1/1 [00:00<00:00,  2.41it/s][['William Shakespeare', 'Romeo and Juliet'], ['Julius Caesar', 'William Shakespeare']]
0.5657978057861328



In [6]:
model = biencoder
compiler_options = "-O2"
optimize = "aggressive"

module_neuron = torch.neuron.trace(model, example_inputs=inputs, compiler_args=compiler_options, optimize = optimize)
#model_neuron = torch.neuron.script(model, compiler_args=compiler_options, optimize = optimize)

NameError: name 'biencoder' is not defined

In [32]:
module_neuron.save("blink_enlity_linking_neuron.pt")

In [17]:
module_loaded_neuron=torch.jit.load("blink_enlity_linking_neuron.pt")

In [18]:
models_jit_neuron = module_loaded_neuron, biencoder_params, crossencoder, crossencoder_params, candidate_encoding, title2id, id2title, id2text, wikipedia_id2local_id, faiss_indexer 

In [ ]:
start = time.time()

_, _, _, _, _, predictions, scores, = main_dense.run(args, None, *models_jit_neuron, test_data=data_to_link, REL_filter=True)
print(predictions)

end = time.time()
print(end - start)